<a href="https://colab.research.google.com/github/amitadhainje/CodingAssignment-UBS/blob/master/Bert_SentimentAnalysis_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing the required modules
!pip install bert-for-tf2
!pip install sentencepiece
print ("The modules like bert-for-tf2 and sentencepiece is installed")

     |████████████████████████████████| 40kB 2.7MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.13.5-cp36-none-any.whl size=29948 sha256=fd2c82c3e844394950c9d984d2064cfff4ecac4dec770022d920badc014af8d0
  Stored in directory: /root/.cache/pip/wheels/ed/11/48/86aff8a0442aa4728c921027a4100ac28878e9d977e76e2d52
  Created wheel for py-params: filename=py_params-0.8.3-cp36-none-any.whl size=4690 sha256=278941524f60daed7244b32fde828541142b88141be76ee000eaea97588dc319
  Stored in directory: /root/.cache/pip/wheels/1a/98/f8/502fc6b5a7c969276f611efeef0d074c871092d38672203b27
  Created wheel for params-flow: filename=params_flow-0.7.4-cp36-none-any.whl size=16195 sha256=c9c245803ce263b89b1ab32f292ea78bb8b996326507fda4956eed497a3c2c4c
  Stored in directory: /root/.cache/pip/wheels/86/30/40/507b60d68b67ac87f35e95c98f5b296a32f146d5ae1d1d5aa7
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.0MB 4.7MB/s 
The modules like bert-for-tf2 and s

In [0]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

In [0]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [28]:
print ("The tensorflow version - ",tf.__version__)
print ("The tensorflow_hub  version - ",hub.__version__)
print ("The pandas version - ",pd.__version__)
print ("The numpy version - ",np.__version__)


The tensorflow version -  2.1.0
The tensorflow_hub  version -  0.7.0
The pandas version -  0.25.3
The numpy version -  1.18.1


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
columns = ['sentiment', 'id', 'date', 'query','user', 'text']
trainData = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train.csv", header=None, names=columns,
                   engine="python", encoding='latin1')

#positiveData = dataset[dataset['sentiment'] == 4][0:50000]
#negativeData = dataset[dataset['sentiment'] == 0][0:50000]
#frames = [positiveData, negativeData]
#trainData = pd.concat(frames)
trainData.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [31]:
trainData.drop(['id','date','query','user'], axis=1, inplace=True)
trainData.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [32]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, 'lxml').get_text()
    tweet = re.sub(r"@[A-Z-a-z-0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Z-a-z-0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z0-9.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

data_clean = [clean_tweet(tweet) for tweet in trainData.text]
data_clean[0]

" Awww that's a bummer. You shoulda got David Carr of Third Day to do it. D"

In [0]:
data_labels = trainData.sentiment.values
data_labels[data_labels == 4] = 1

In [0]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1', trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [0]:
def encode_sentence(sent):
    return ['[CLS]'] + tokenizer.tokenize(sent) + ['[SEP]']

data_inputs = [encode_sentence(sentence) for sentence in data_clean]

1. Add "CLS" and "SEP" tokens to the starting and end of the sequence. Here we are assuming that there is only one sequence.

2. A "BERT_LAYER" takes 3 different inputs.

3. First is the tokenized version of the input sentences i.e. the one we have created in using the above "encode_sentence" function. Second is the list of the masks or "input_masks" which helps bert to differentiate between the actual values and the padding values. It tells the bert where are the padding values for a given sentence. The padded tokens should not be used while processing. Third one is the segment ids , which will be used if there are multiple sentences in a particular records. Segment ids are combination of '0's and '1's and they tell the bert which is the first sentence and which one is the second sentence in one particular record.

In [0]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id
    return seg_ids





In [0]:
# The training data is sorted according to the target variable, hence we are going to shuffle the data
import random
data_with_len = [[sent,data_labels[i], len(sent)]
                 for i,sent in enumerate(data_inputs)]

random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]), 
               get_mask(sent_lab[0]),
                get_segments(sent_lab[0]) 
               ], sent_lab[1])   
                for sent_lab in data_with_len if sent_lab[2] > 7 ]